In [ ]:
import pandas as pd
from sqlite3 import connect

def IoTcode2vulBERT(csv_input, csv_output):
    df = pd.read_csv(csv_input)
    """ convert data to vulBERT format """
    if 'cwe' in list(df.columns):
        df = df[['code', 'is_vul']]
        df = df.rename(columns={'is_vul':'label'})
        df['label'] = df.label.astype(int)

    df.to_csv(csv_output)
    return df

dataset = 'raspberryZip'
csv_input = f"../data/{dataset}_function.csv"
csv_vulbert =f"../data/{dataset}_function-VulBERT.csv"
# csv = '../VulBERTa/data/finetune/d2a/function/d2a_lbv1_function_dev.csv'

# df = IoTcode2vulBERT(
#     csv_input=csv_input,
#     csv_output=csv_vulbert
#     )

In [ ]:
dataset = '../data/IoTcode_contiki_rasp_filtered_full.db'
conn = connect(dataset)

df_fun = pd.read_sql(sql="SELECT * FROM function", con=conn)
df_stat = pd.read_sql(sql="SELECT * FROM statement", con=conn)
df_prj = pd.read_sql(sql="SELECT * FROM project", con=conn)

In [ ]:
pd.read_sql(sql="SELECT vul, count(*) FROM function GROUP BY project;", con=conn)

In [ ]:
dataset = 'raspberryZip'
flaw = f"../data/{dataset}_statement.csv"
metrics = f"../data/{dataset}_function.csv"
binary = f"../data/{dataset}_method_binary.csv"

df_stat  = pd.read_csv(flaw)
df_fun = pd.read_csv(metrics)

print(f'Fun statistics: \n{df_fun.isVul.value_counts()}')
print(f'\nStat statistics: \n{df_stat.cwe.value_counts()}')

In [ ]:
df_fun.drop_duplicates(subset=['filename', 'long_name', 'start_line', 'end_line', 'cwe'], keep='last').reset_index(drop=True)

In [ ]:
print(len(df_stat.columns)) 
df_stat = df_stat.dropna(axis=1, how='all')
print(len(df_stat.columns))
df_stat

In [ ]:
str(df_fun[df_fun.cwe=="['CWE-120', 'CWE-120, CWE-20']"]['filename'])

In [ ]:
import ruamel

# yaml = yaml.YAML()
# config = yaml.safe_load(open("../ext_projects.yaml"))
with open("../ext_projects.yaml") as f:
        # config = yaml.load(f, Loader=yaml.RoundTripLoader)
        doc = ruamel.yaml.load(f.read(), Loader=ruamel.yaml.RoundTripLoader)
        doc['save']['drop_dup'] = '10'
        print(ruamel.yaml.dump(doc, Dumper=ruamel.yaml.RoundTripDumper))

ruamel.yaml.preserve_quotes = True
config['save']['drop_dup'] = '10'
print(config)

# with open('file_to_edit.yaml', 'w') as f:
#         yaml.dump(config, f)

In [ ]:
import ruamel
import sys

# yaml = yaml.YAML()
# config = yaml.safe_load(open("../ext_projects.yaml"))
with open("../ext_projects.yaml") as f:
    yaml = ruamel.yaml.YAML()
    yaml.indent(mapping=4, sequence=4, offset=1)
    yaml.preserve_quotes = True
    params = yaml.load(f.read())
    params['projects'].yaml_add_eol_comment('some comment', key='new_key', column=40)
    config['save']['drop_dup'] = '10'
    # params['ParentTest']['test'].yaml_add_eol_comment('some comment', key='new_key', column=40) # column is optional
    yaml.dump(params, sys.stdout)

In [ ]:
import time
from humanfriendly import format_timespan
start_time = time.time()

print("--- %s ---" % format_timespan(time.time() - start_time))

In [ ]:
import pandas as pd

%run ../src/sqlite.py
db = Database("../data/iotcode.db")
from sqlite import Database

df = pd.read_sql(f"SELECT * FROM statement", con=db.conn)
df.head()